In [1]:
!pip install yfinance langchain fastapi google.generativeai dotenv  --quiet


[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: C:\Users\HARI PRIYA\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [2]:
import requests
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display
from io import BytesIO
import os
#from google.colab import files
from IPython.display import display, Image
import base64

- P/E Ratio 📈 (How overpriced this stock is?)

- Debt-to-Equity Ratio ⚖️ (Is this company running on loans?)

- EPS 💰 (Does this stock print money?)

- Dividend Yield 📦 (Does this stock reward its cult members?)

- Trailing PEG Ratio 🏃 (Growth-adjusted valuation?)

- Payout Ratio 🎁 (What % of earnings go to dividends?)

- Beta ⚡ (Is this stock a rollercoaster?)

- Volume & Avg Volume 📊 (How much this stock gets traded?)

- 52-Week Low & 50-Day Average 📉 (How beaten down it is?)

- Profit Margins 💵 (Does this company even make money?)

- Short Ratio 🏴‍☠️ (How many people are betting it dies?)

- Enterprise-to-Revenue & EBITDA 🏢 (How efficient is the company?)

- Revenue Growth 🚀 (Is it expanding or dying?)



####Train model on this and lstm on previous stock prices using yfinance(3months would be enough). It will be a classification problem whether to buy or sell or hold.
- Also make sure that we keep a front end to download this markdown file
- Hey wait, can also make a markdown viewer right? Lets do that first then integrate it with this so that we get the report in the website only

- ORRR we can just create various tags and then say XYZ metric: value and then Final Verdict: BUY/SELL/HOLD

- Later versions - Chatbot to tell why that thing is good.


In [7]:
import requests
import yfinance as yf
import matplotlib.pyplot as plt
import os
import base64
from io import BytesIO
from dotenv import load_dotenv
import google.generativeai as genai

# Load API keys from .env file
load_dotenv()
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
GENAI_API_KEY = os.getenv("GENAI_API_KEY")

# Configure Gemini AI
if GENAI_API_KEY:
    genai.configure(api_key=GENAI_API_KEY)
    model = genai.GenerativeModel("gemini-2.0-flash")
else:
    model = None
# # Set up directory
# REPORTS_DIR = "reports"
# os.makedirs(REPORTS_DIR, exist_ok=True)

def get_stock_history_alpha_vantage(ticker):
    """Fetch detailed historical fundamental data from Alpha Vantage"""
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={ALPHA_VANTAGE_API_KEY}"
    response = requests.get(url)
    data = response.json()

    return {
        "P/E Ratio": float(data.get("PERatio", 0)),
        "Debt-to-Equity": float(data.get("DebtEquityRatio", 0)),
        "EPS": float(data.get("EPS", 0)),
        "Dividend Yield (%)": float(data.get("DividendYield", 0)) * 100,
        "Profit Margins (%)": float(data.get("ProfitMargin", 0)) * 100,
        "Revenue Growth (%)": float(data.get("QuarterlyRevenueGrowthYOY", 0)) * 100,
        "Return on Assets (%)": float(data.get("ReturnOnAssetsTTM", 0)) * 100,
        "Return on Equity (%)": float(data.get("ReturnOnEquityTTM", 0)) * 100,
        "Operating Margin (%)": float(data.get("OperatingMarginTTM", 0)) * 100,
        "Beta": float(data.get("Beta", 0)),
        "Book Value per Share": float(data.get("BookValue", 0)),
        "Market Cap (in Billion $)": float(data.get("MarketCapitalization", 0)) / 1e9,
        "EV to EBITDA": float(data.get("EVToEBITDA", 0)),
        "EV to Revenue": float(data.get("EVToRevenue", 0)),
        "Revenue Per Share": float(data.get("RevenuePerShareTTM", 0)),
        "Quarterly Earnings Growth YOY": float(data.get("QuarterlyEarningsGrowthYOY", 0)),
        "PEGRatio (%)": float(data.get("PEGRatio", 0))
    }



def get_stock_current_yfinance(ticker):
    """Fetch current fundamental data from Yahoo Finance"""
    stock = yf.Ticker(ticker)
    info = stock.info

    return {
        "P/E Ratio": info.get("trailingPE", 0),
        "Debt-to-Equity": info.get("totalDebt", 1) / (info.get("totalStockholderEquity", 1) or 1),
        "EPS": info.get("trailingEps", 0),
        "Dividend Yield (%)": info.get("dividendYield", 0) * 100,
        "Profit Margins (%)": info.get("profitMargins", 0) * 100,
        "Revenue Growth (%)": info.get("revenueGrowth", 0) * 100,
        "Beta": info.get("beta", 0),
        "Short Ratio": info.get("shortRatio", 0),
        "Enterprise to Revenue": info.get("enterpriseToRevenue", 0),
        "Enterprise to EBITDA": info.get("enterpriseToEbitda", 0)
    }
def sanitize_filename(metric):
    """Replace invalid filename characters"""
    return metric.replace(" ", "_").replace("/", "-")
def generate_ai_insights(ticker, current_data, historical_data):
    if not model:
        return "AI insights unavailable. Set up GENAI_API_KEY in .env."
    
    prompt = f"""
    You are a professional financial analyst writing an AI-generated investment report on {ticker}.

    ### **📊 Company Overview**
    Analyze the company’s financial health using the latest data.

    ### **🔹 Key Observations**
    Summarize important financial trends, revenue growth, and stock movement.

    ### **⚠️ Risk Analysis**
    Highlight potential risks, including market volatility, debt levels, and economic factors.

    ### **📈 Growth Potential**
    Evaluate opportunities for expansion, innovation, or sector advantage.

    ### **📌 Final Investment Outlook**
    Give a clear, concise recommendation: Buy, Hold, or Sell. Justify your decision with key insights.

    ⏳ Be factual, to the point, and professional. Keep responses under 200 words.
    """
    response = model.generate_content(prompt)
    return response.text.strip()

def generate_charts(ticker, current_data, historical_data):
    """Generate and save line charts for each financial metric"""
    reports_dir = "reports"
    os.makedirs(reports_dir, exist_ok=True)  # Ensure the directory exists
    chart_paths = {}

    for metric, value in current_data.items():
        sanitized_metric = sanitize_filename(metric)
        values = [historical_data.get(metric, 0), value]
        labels = ["Historical", "Current"]

        plt.figure(figsize=(5, 3))
        plt.plot(labels, values, marker='o', linestyle='-', color='b')
        plt.xlabel("Time Period")
        plt.ylabel(metric)
        plt.title(f"{ticker} - {metric} Over Time")
        plt.grid(True, linestyle="--", alpha=0.6)

        buf = BytesIO()
        plt.savefig(buf, format="png", bbox_inches="tight")
        buf.seek(0)
        chart_paths[metric] = base64.b64encode(buf.read()).decode("utf-8")
        plt.close()

    return chart_paths

def encode_image_to_base64(image_path):
    
    with open(image_path, "rb") as img_file:
        
        return base64.b64encode(img_file.read()).decode("utf-8")

def generate_markdown_report(ticker):
    """Generate AI-powered financial report with uncommon insights"""
    current_data = get_stock_current_yfinance(ticker)
    historical_data = get_stock_history_alpha_vantage(ticker)
    chart_paths = generate_charts(ticker, current_data, historical_data)
    
    
    md_report = f"# 📊 Investment Report: {ticker}\n\n"
    md_report += "## 📜 Table of Contents\n"
    md_report += "- [🔍 Current Financial Overview](#🔍-current-financial-overview)\n"
    md_report += "- [📅 Historical Comparison](#📅-historical-comparison)\n"
    md_report += "- [📊 Key Insights & Trends](#📊-key-insights--trends)\n"
    md_report += "- [📈 Charts](#📈-charts)\n\n"
    
    
    md_report += "## 🔍 Current Financial Overview\n\n"
    md_report += "| Metric | Value |\n"
    md_report += "|--------|-------|\n"
    for metric, value in current_data.items():
      md_report += f"| **{metric}** | {value:.2f} |\n"
    md_report += "\n"
    
    md_report += "\n## 📅 Historical Comparison\n"
    
    for metric in historical_data.keys():
      historical_value = historical_data.get(metric, 0)
      trend = (
          "<span style='color:green'>📈 Increasing</span>" if metric in current_data and historical_value < current_data[metric]
          else "<span style='color:red'>📉 Decreasing</span>" if metric in current_data and historical_value > current_data[metric]
          else "<span style='color:gray'>⚖️ No Change</span>"
      )
      md_report += f"- **{metric}** ({trend})\n"
      md_report += f"  - **Past Value:** {historical_value:.2f}, **Current Value:** {current_data.get(metric, historical_value):.2f}\n"
    
    md_report += "\n---\n"
    md_report += "### 📊 AI Insights\n"
    ai_insights = generate_ai_insights(ticker, current_data, historical_data)
    md_report+=f"{ai_insights}\n"
    
    md_report += "\n---\n"
    md_report += "### 🚀 Key Insights & Trends\n"
    
    if current_data["P/E Ratio"] > 50:
      md_report += "- 🚨 **High P/E Ratio**: Stock may be overvalued or experiencing extreme speculation.\n"
    if current_data["Profit Margins (%)"] < 0:
      md_report += "- ❌ **Negative Profit Margins**: The company is operating at a loss.\n"
    if current_data["Debt-to-Equity"] > 2:
      md_report += "- ⚠️ **High Debt Load**: This company is carrying substantial financial risk.\n"
    
    md_report += "\n---\n### 🛸 Uncommon Market Insights\n"
    
    # Extreme Volatility (Beta > 2)
    if current_data["Beta"] > 2:
      md_report += "- 🌪️ **Extreme Volatility**: This stock has high market fluctuations.\n"
    
    # Short Interest (Short Ratio > 5)
    if current_data["Short Ratio"] > 5:
      md_report += "- 🐻 **High Short Interest**: Many traders are betting against this stock.\n"
    
    # High Enterprise to Revenue Ratio (Possible Overvaluation)
    if current_data["Enterprise to Revenue"] > 10:
      md_report += "- 🏢 **Expensive Stock**: The enterprise valuation is significantly higher than revenue.\n"
    for metric, base64_img in chart_paths.items():  # Now directly contains base64
        md_report += f"- **{metric}**\n  - ![Chart](data:image/png;base64,{base64_img})\n"
    
    
    
    md_report += "\n---\n"
    report_path = os.path.join(REPORTS_DIR, f"{ticker}_investment_report.md")
    with open(report_path, "w",encoding="utf-8") as f:
         f.write(md_report)
    print(f"✅ Report saved: {report_path}")
    return report_path
    
if __name__ == "__main__":
    ticker = "AAPL"
    report_path = generate_markdown_report(ticker)
    print(f"Report generated at: {report_path}")


✅ Report saved: reports\AAPL_investment_report.md
Report generated at: reports\AAPL_investment_report.md
